In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pickle

In [2]:
datatset = pd.read_csv('../data/merged_data.csv')

In [3]:
datatset.head()

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,DAYS_EMPLOYED,OCCUPATION_TYPE,CNT_FAM_MEMBERS,APPROVED
0,5008804,Y,Y,427500.0,Working,Higher education,Rented apartment,-4542,Other,2.0,1
1,5008806,Y,Y,112500.0,Working,Secondary / secondary special,House / apartment,-1134,Security staff,2.0,1
2,5008808,N,Y,270000.0,Commercial associate,Secondary / secondary special,House / apartment,-3051,Sales staff,1.0,1
3,5008812,N,Y,283500.0,Pensioner,Higher education,House / apartment,365243,Other,1.0,0
4,5008815,Y,Y,270000.0,Working,Higher education,House / apartment,-769,Accountants,2.0,0


In [4]:
datatset = datatset.drop('ID', axis=1)

In [5]:
train_set, test_set = train_test_split(datatset, test_size=0.25, random_state=42)

In [6]:
scaler = StandardScaler()
ordinal_encoder = OrdinalEncoder()
onehot_encoder = OneHotEncoder()

In [7]:
train_set_encoded = pd.get_dummies(train_set, columns=['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE'], dtype=int)
test_set_encoded = pd.get_dummies(test_set, columns=['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE'], dtype=int)

In [8]:
train_set_encoded['FLAG_OWN_CAR'] = train_set_encoded['FLAG_OWN_CAR'].replace({'Y': 1, 'N': 0})
train_set_encoded['FLAG_OWN_REALTY'] = train_set_encoded['FLAG_OWN_REALTY'].replace({'Y': 1, 'N': 0})

In [9]:
test_set_encoded['FLAG_OWN_CAR'] = test_set_encoded['FLAG_OWN_CAR'].replace({'Y': 1, 'N': 0})
test_set_encoded['FLAG_OWN_REALTY'] = test_set_encoded['FLAG_OWN_REALTY'].replace({'Y': 1, 'N': 0})

In [10]:
train_set_encoded[['AMT_INCOME_TOTAL', 'DAYS_EMPLOYED']] = scaler.fit_transform(train_set_encoded[['AMT_INCOME_TOTAL', 'DAYS_EMPLOYED']])

In [11]:
test_set_encoded[['AMT_INCOME_TOTAL', 'DAYS_EMPLOYED']] = scaler.fit_transform(test_set_encoded[['AMT_INCOME_TOTAL', 'DAYS_EMPLOYED']])

In [12]:
train_set_encoded[['CNT_FAM_MEMBERS']] = ordinal_encoder.fit_transform(train_set_encoded[['CNT_FAM_MEMBERS']])

In [13]:
test_set_encoded[['CNT_FAM_MEMBERS']] = ordinal_encoder.fit_transform(test_set_encoded[['CNT_FAM_MEMBERS']])

In [14]:
train_set_encoded

,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,DAYS_EMPLOYED,CNT_FAM_MEMBERS,APPROVED,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Other,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff
6685,0,0,0.905732,-0.449253,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2598,0,0,-0.011864,-0.445646,3.0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
102,1,0,-0.470662,-0.439362,1.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5896,1,0,-0.700061,2.190542,1.0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4385,1,1,-0.378902,-0.461123,1.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0,0,-0.470662,-0.459719,1.0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5191,1,1,-0.011864,-0.457668,1.0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5226,0,1,0.217535,-0.453413,1.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5390,0,0,-0.241263,-0.477060,1.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
classifier = RandomForestClassifier(criterion="entropy")
classifier.fit(train_set_encoded.drop(columns=['APPROVED'], axis=1), train_set_encoded[['APPROVED']])

C:\Users\ADMIN\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(criterion='entropy')

In [16]:
accuracy_score(test_set_encoded[['APPROVED']], classifier.predict(test_set_encoded.drop(columns=['APPROVED'], axis=1)))

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- NAME_INCOME_TYPE_Student


In [ ]:
filename = '../output_model/model.pkl'
pickle.dump(classifier, open(filename, 'wb'))